# Memory

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.chat_history import BaseChatMessageHistory

class InMemoryHistory(BaseChatMessageHistory):
    messages = []

    def add_messages(self, messages):
        self.messages.extend(messages)


    def clear(self):
        self.messages = []

    def __repr__(self):
        return str(self.messages)


In [3]:
store = {}      # item(key=session_id, value=InMemoryHistory_인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [7]:
history_test = get_by_session_id('test')
history_test.add_messages(['hello', 'good morning', 'how are you?'])
history_test.add_messages(['I am fine', 'Thank you?'])

history_test

['heelo', 'good morning', 'how are you?', 'heelo', 'good morning', 'how are you?', 'I am fine', 'Thank you?', 'hello', 'good morning', 'how are you?', 'I am fine', 'Thank you?']